# Software Development

While the IoT kernel won't replace an IDE, it contains a few features that support code development. In addition to [software upload](file_ops.ipynb) (`%rsync` magic), the

* `%softreset` (same as ^D in repl),
* `%upip` and `%pip` package managers, and
* `%mpycross` cross-compiler

come in handy.

In [6]:
%connect -s my_esp

Connected to my_esp @ serial:///dev/cu.usbserial-0163A3DA


## %softreset

Softreset restarts the MicroPython interpeter, clearing all variables and releasing peripherals. The latter is particularly convenient with CircuitPython as an alternative to using context managers during code development.

*Example:*

In [19]:
a = 5
print(a)
%softreset
print(a)   # raises NameError

5

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
!!!!!   softreset ...     !!!!!
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!



Traceback (most recent call last):
  File "<stdin>", line 1, in <module>
NameError: name 'a' isn't defined


## Package managers: %upip and %pip

Two versions of the pip magic are provided: 

* `%pip` uses "standard" pip to download packages from PyPi; e.g. I2C drivers from [Adafruit](https://circuitpython.readthedocs.io/projects/bundle/en/latest/drivers.html). Although targeted for CircuitPython, many of these libraries can also be used on MicroPython with simple adaptor code.
   
* `%upip` downloads MicroPython libraries from https://micropython.org. See https://github.com/micropython/micropython-lib for what's available. Note that MicroPython uses a special package format (that is incompatible with standard pip), hence you need to use `%upip` to download these libraries.

#### %pip

The magic below fetches the driver for BME280 temperature/pressure sensor.

In [6]:
%pip install Adafruit-BME280
%rsync

  Using cached Adafruit_BME280-1.0.1-py3-none-any.whl (6.3 kB)
COPY    lib/Adafruit_BME280/BME280.py
COPY    lib/Adafruit_BME280/__init__.py


Note that the code is only downloaded to the host; use the `%rsync` macro to upload to the microcontroller.

#### %upip

*Example:* download & install the logging module:

In [6]:
%upip install logging
%rsync

Installing micropython-logging 0.3 from https://micropython.org/pi/logging/logging-0.3.tar.gz
COPY    lib/logging.py


## %mpycross

In [6]:
%mpycross

compiling   boards/esp32/mcu/webrepl_cfg.py
compiling   boards/esp32/mcu/boot.py
compiling   boards/esp32/mcu/lib/logging.py
compiling   boards/esp32/mcu/lib/utelnetserver.py
compiling   boards/esp32/mcu/lib/Adafruit_BME280/BME280.py
compiling   boards/esp32/mcu/lib/Adafruit_BME280/__init__.py
compiling   boards/esp32/mcu/lib/mp/server.py
compiling   boards/esp32/mcu/lib/mp/__init__.py
compiling   boards/esp32/mcu/lib/ota32/open_url.py
compiling   boards/esp32/mcu/lib/ota32/__init__.py
compiling   boards/esp32/mcu/lib/ota32/ota.py


The next sync uploads the compiled files and deletes the sources. The sources are still available on the host, of course (the IoT Kernel never deleteds files on the host).

In [6]:
%rsync
%rlist

DELETE  webrepl_cfg.py
DELETE  lib/utelnetserver.py
DELETE  lib/ota32/ota.py
DELETE  lib/ota32/open_url.py
DELETE  lib/ota32/__init__.py
DELETE  lib/mp/server.py
DELETE  lib/mp/__init__.py
DELETE  lib/logging.py
DELETE  lib/Adafruit_BME280/__init__.py
DELETE  lib/Adafruit_BME280/BME280.py
DELETE  boot.py
COPY    webrepl_cfg.mpy
COPY    lib/mp/__init__.mpy
COPY    lib/ota32/open_url.mpy
COPY    lib/ota32/ota.mpy
COPY    lib/ota32/__init__.mpy
COPY    lib/utelnetserver.mpy
COPY    lib/mp/server.mpy
COPY    lib/Adafruit_BME280/__init__.mpy
COPY    lib/logging.mpy
COPY    lib/Adafruit_BME280/BME280.mpy
COPY    boot.mpy
    878  Feb 11 16:52 2021  boot.mpy
                            lib/
                                Adafruit_BME280/
   5085  Feb 11 16:52 2021          BME280.mpy
     45  Feb 11 16:52 2021          __init__.mpy
   1228  Feb 11 16:52 2021      logging.mpy
                                mp/
     83  Feb 11 16:52 2021          __init__.mpy
   2291  Feb 11 16:52 2021       

Modifying a .py file results in the newer file to be uploaded. 

In [7]:
!touch boards/esp32/mcu/lib/logging.py
%rsync

DELETE  lib/logging.mpy
COPY    lib/logging.py


Run %mpycross again to compile ...

In [7]:
%mpycross
%rsync

compiling   boards/esp32/mcu/lib/logging.py
DELETE  lib/logging.py
COPY    lib/logging.mpy


# Software Development

Jupyter can be used to prototype and design code.

*Example:* TimeIt ... measure execution time. 

Iteratively write the class, test, add features, retest ... right from the noteboook.

In [11]:
# after development is complete, 
# uncomment next line to write file to disk

# %%writefile boards/esp32/mcu/lib/timeit.py

from utime import ticks_ms, ticks_diff

class TimeIt:
    
    def __init__(self):
        self.reset()
        
    @property
    def elapsed_ms(self) -> int:
        """Elapsed time in ms."""
        return ticks_diff(ticks_ms(), self.start)
    
    @property
    def elapsed(self) -> float:
        """Elapsed time in seconds."""
        return self.elapsed_ms / 1000
    
    def reset(self):
        self.start = ticks_ms()
        
    def __enter__(self):
        self.reset()
        return self
    
    def __exit__(self, *args):
        pass

Overwriting boards/esp32/mcu/lib/timeit.py


In [11]:
# testing
    
from time import sleep

with TimeIt() as t:
    sleep(1.2345)
    print("elapsed [ms]:", t.elapsed_ms)   
    print("elapsed [s] :", t.elapsed)   

elapsed [ms]: 1234
elapsed [s] : 1.235


When satisfied, uncomment the `%%writefile` magic and re-run the cell to commit it to the disk.

In [11]:
%rsync

%cat lib/timeit.py

DELETE  lib/timeit.mpy
COPY    lib/timeit.py

from utime import ticks_ms, ticks_diff

class TimeIt:
    
    def __init__(self):
        self.reset()
        
    @property
    def elapsed_ms(self) -> int:
        """Elapsed time in seconds."""
        return ticks_diff(ticks_ms(), self.start)
    
    @property
    def elapsed(self) -> float:
        """Elapsed time in seconds."""
        return self.elapsed_ms / 1000
    
    def reset(self):
        self.start = ticks_ms()
        
    def __enter__(self):
        self.reset()
        return self
    
    def __exit__(self, *args):
        pass



In [11]:
from timeit import TimeIt
import time

with TimeIt() as t:
    time.sleep(0.472)
    print("execution time:", t.elapsed)

execution time: 0.472


Optionally compile ...

In [10]:
%mpycross
%rsync

!ls -l boards/esp32/mcu/lib/timeit.py
!ls -l .boards/esp32/mcu-micropython/lib/timeit.mpy

compiling   boards/esp32/mcu/lib/timeit.py
DELETE  lib/timeit.py
COPY    lib/timeit.mpy
-rw-r--r--@ 1 boser  staff  549 Feb 11 17:25 boards/esp32/mcu/lib/timeit.py
-rw-r--r--@ 1 boser  staff  308 Feb 11 17:26 .boards/esp32/mcu-micropython/lib/timeit.mpy


The compiled class is about a third smaller.


----

#### restore original content ...

In [ ]:
!rm -rf $IOT49/boards/esp32/mcu/lib/timeit.py 
!rm -rf $IOT49/boards/esp32/mcu/lib/logging.py 
!rm -rf $IOT49/boards/esp32/mcu/lib/Ada* 
!rm -rf $IOT49/.boards
%rsync